# Testing CLAIM Models

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing out EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

In [1]:
import numpy as np
import pandas as pd
import torch

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import claim.modules as cm

# Load data

In [69]:
x = torch.randn(10, 4, 66)
x_t = x.transpose(2, 1)
x_rev = torch.randn(10, 4, 66)
x_rev_t = x_rev.transpose(2, 1)

# FCN Only

In [4]:
from eugene.models.FCN import FCN

In [34]:
fcn = FCN(input_len=66, 
          strand="ts",
          task="binary_classification",
          fc_kwargs=dict(hidden_dims=[100], output_dim=1))
fcn

FCN(
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=100, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=100, out_features=1, bias=True)
    )
  )
  (reverse_fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=100, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=100, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [37]:
out = fcn(x, x_rev)
out.shape

tensor([[-0.3729,  0.1600],
        [ 0.2604, -0.3764],
        [ 0.1441, -0.0135],
        [-0.2043, -0.4903],
        [ 0.0234, -0.0656],
        [-0.1336, -0.0384],
        [ 0.1261, -0.2827],
        [ 0.2611, -0.1058],
        [-0.2962, -0.4841],
        [ 0.0235, -0.2443]], grad_fn=<CatBackward0>)


torch.Size([10, 1])

In [38]:
out

tensor([[-0.1065],
        [-0.0580],
        [ 0.0653],
        [-0.3473],
        [-0.0211],
        [-0.0860],
        [-0.0783],
        [ 0.0776],
        [-0.3901],
        [-0.1104]], grad_fn=<UnsqueezeBackward0>)

# CNN Only

In [52]:
from eugene.models.CNN import CNN

<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of a CNN model</b>
</div>

In [53]:
cnn = CNN(input_len=66, 
          strand="ts",
          task="binary_classification", 
          conv_kwargs=dict(channels=[4, 16, 32, 64], conv_kernels=[15, 5, 5], pool_kernels=[1, 1, 1], pool_strides = [1, 1, 1], dropout_rates=0.2), 
          fc_kwargs=dict(output_dim=1))
cnn

CNN(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
      (8): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
      (9): ReLU(inplace=True)
      (10): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (11): Dropout(p=0.2, inplace=False)
    )
  )
  (reverse_convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv

In [54]:
out = cnn(x, x_rev)
out.shape

torch.Size([10, 1])

# RNN Only

In [55]:
from eugene.models.RNN import RNN

<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of a RNN model</b>
</div>

In [71]:
rnn = RNN(input_len=66, 
          strand="ds",
          task="regression",
          rnn_kwargs=dict(output_dim=32, unit_type="lstm", bidirectional=True, batch_first=True), 
          fc_kwargs=dict(output_dim=1))
rnn

RNN(
  (rnn): BasicRecurrent(
    (module): LSTM(4, 32, batch_first=True, bidirectional=True)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [72]:
out = rnn(x_t, x_rev_t)
out.shape

torch.Size([10, 1])

# Hybrid CNN-RNN

In [73]:
from eugene.models.hybrid import hybrid

<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of a CNN model</b>
</div>

In [81]:
cnn_rnn = hybrid(input_len=66, 
                 strand="ts",
                 task="binary_classification", 
                 conv_kwargs=dict(channels=[4, 16, 32], conv_kernels=[15, 5], pool_kernels=[1, 1, 1], pool_strides = [1, 1], dropout_rates=0.2), 
                 rnn_kwargs=dict(output_dim=32, unit_type="lstm", bidirectional=True, batch_first=True),  
                 fc_kwargs=dict(output_dim=1))
cnn_rnn

hybrid(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (reverse_convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (r

In [82]:
out = cnn_rnn(x, x_rev)
out.shape

torch.Size([10, 1])

# Scratch